In [13]:
import random
import sys
import getopt
from PIL import Image, ImageDraw, ImageFont, ImageTk
from math import sqrt
import logging
from IPython import display
import time
import tkinter as tk

In [14]:
def all_pairs(size,shuffle=random.shuffle):
    '''generates all i,j pairs for i,j from 0-size uses shuffle to randomise (if provided)'''
    r1=list(range(size))
    r2=list(range(size))
    if shuffle:
        shuffle(r1)
        shuffle(r2)
    for i in r1:
        for j in r2:
            yield (i,j)

In [15]:
def reversed_sections(tour):
    '''generator to return all possible variations where the section between two cities are swapped'''
    for i,j in all_pairs(len(tour)):
        if i != j:
            copy=tour[:]
            if i < j:
                copy[i:j+1]=reversed(tour[i:j+1])
            else:
                copy[i+1:]=reversed(tour[:j])
                copy[:j]=reversed(tour[i+1:])
            if copy != tour: # no point returning the same tour
                yield copy

def swapped_cities(tour):
    '''generator to create all possible variations where two cities have been swapped'''
    for i,j in all_pairs(len(tour)):
        if i < j:
            copy=tour[:]
            copy[i],copy[j]=tour[j],tour[i]
            yield copy

In [16]:
def cartesian_matrix(coords):
    '''create a distance matrix for the city coords that uses straight line distance'''
    matrix={}
    for i,(x1,y1) in enumerate(coords):
        for j,(x2,y2) in enumerate(coords):
            dx,dy=x1-x2,y1-y2
            dist=sqrt(dx*dx + dy*dy)
            matrix[i,j]=dist
    return matrix


In [17]:
def tour_length(matrix,tour):
    '''total up the total length of the tour based on the distance matrix'''
    total=0
    num_cities=len(tour)
    for i in range(num_cities):
        j=(i+1)%num_cities
        city_i=tour[i]
        city_j=tour[j]
        total+=matrix[city_i,city_j]
    return total

In [18]:
def tour_to_img(coords,tour,title):
    padding=20
    # shift all coords in a bit
    coords=[(x+padding,y+padding) for (x,y) in coords]
    maxx,maxy=0,0
    for x,y in coords:
        maxx=max(x,maxx)
        maxy=max(y,maxy)
    maxx+=padding
    maxy+=padding
    img=Image.new("RGB",(int(maxx),int(maxy)),color=(255,255,255))
    
    font=ImageFont.load_default()
    d=ImageDraw.Draw(img)
    num_cities=len(tour)
    for i in range(num_cities):
        j=(i+1)%num_cities
        city_i=tour[i]
        city_j=tour[j]
        x1,y1=coords[city_i]
        x2,y2=coords[city_j]
        d.line((int(x1),int(y1),int(x2),int(y2)),fill=(0,0,0))
        d.text((int(x1)+7,int(y1)-5),str(i),font=font,fill=(32,32,32))
    
    
    for x,y in coords:
        x,y=int(x),int(y)
        d.ellipse((x-5,y-5,x+5,y+5),outline=(0,0,0),fill=(196,196,196))
    
    d.text((1,1),title,font=font,fill=(0,0,0))
    
    del d
    #img.save()
    return img

In [19]:
def init_random_tour(tour_length):
   tour=list(range(tour_length))
   random.shuffle(tour)
   return tour

In [20]:
def read_coords(coord_file):
    '''
    read the coordinates from file and return the distance matrix.
    coords should be stored as comma separated floats, one x,y pair per line.
    '''
    coords=[]
    for line in coord_file:
        x,y=line.strip().split(',')
        coords.append((float(x),float(y)))
    return coords

In [21]:
def hillclimb(init_function,move_operator,objective_function,max_evaluations,coords):
    '''
    hillclimb until either max_evaluations is reached or we are at a local optima
    '''
    best=init_function()
    best_score=objective_function(best)
    all_img = []
    num_evaluations=1
    
    logging.info('hillclimb started: score=%f',best_score)
    
    while num_evaluations < max_evaluations:
        # examine moves around our current position
        move_made=False
        for next in move_operator(best):
            if num_evaluations >= max_evaluations:
                break
            
            # see if this move is better than the current
            next_score=objective_function(next)
            num_evaluations+=1
            if next_score < best_score:
                best=next
                best_score=next_score
                move_made=True
                out_img=tour_to_img(coords,best,'score : %f'%(best_score))
                all_img.append(out_img)
                
                break # depth first search
            
        if not move_made:
            break # we couldn't find a better move (must be at a local maximum)
    
    logging.info('hillclimb finished: num_evaluations=%d, best_score=%f',num_evaluations,best_score)
    return (num_evaluations,best_score,best,all_img)

def hillclimb_and_restart(init_function,move_operator,objective_function,max_evaluations,coords):
    '''
    repeatedly hillclimb until max_evaluations is reached
    '''
    best=None
    best_score=0
    
    num_evaluations=0
    while num_evaluations < max_evaluations:
        remaining_evaluations=max_evaluations-num_evaluations
        
        logging.info('(re)starting hillclimb %d/%d remaining',remaining_evaluations,max_evaluations)
        evaluated,score,found,all_img=hillclimb(init_function,move_operator,objective_function,remaining_evaluations,coords)
        
        num_evaluations+=evaluated
        if score < best_score or best is None:
            best_score=score
            best=found
        
    return (num_evaluations,best_score,best,all_img)

In [22]:
def run_hillclimb(init_function,move_operator,objective_function,max_iterations,coords):
    iterations,score,best,all_img=hillclimb_and_restart(init_function,move_operator,objective_function,max_iterations,coords)
    return iterations,score,best,all_img

In [23]:
class Image_Displayer:
    def __init__(self, image,score,best):
        self.score = score
        self.best = best
        self.root = tk.Tk()
        self.root.title("shortest path finding")
        self.index = 0
        self._images = image
        image_label = ImageTk.PhotoImage(self._images[self.index])
        self.root.label= tk.Label(image=image_label)
        self.root.label.pack()
        self.root.label.after(50, lambda:self.change_label() )
        
        self.start()
        

    def change_label(self):
        self.index += 1
        if self.index < len(self._images):
            img = ImageTk.PhotoImage(self._images[self.index])
            self.root.label.configure(image=img)
            self.root.label.image = img
            self.root.update_idletasks()
            self.root.label.after(50, lambda:self.change_label())
        else:
            time.sleep(0.07)
            print ( "Best score : %d Best path ="%self.score,self.best)

    def start(self):
        self.root.mainloop()
        
    def destroy(self):
        self.root.destroy()
        self.root.quit()

In [ ]:
 def main():
    city_file="city100.txt"
    out_file_name="result.png"
    max_iterations=100000
    move_operator=reversed_sections


    if out_file_name and not out_file_name.endswith(".png"):

        print ("output image file name must end in .png")
        sys.exit(1)


    # setup the things tsp specific parts hillclimb needs
    f = open(city_file, 'r')
    coords=read_coords(f)
    f.close()

    init_function=lambda: init_random_tour(len(coords))
    matrix=cartesian_matrix(coords)
    objective_function=lambda tour: tour_length(matrix,tour)

    #hill climb
    
    iterations,score,best,all_img=run_hillclimb(init_function,move_operator,objective_function,max_iterations,coords)
    

    #save result to image
    result_img=tour_to_img(coords,best,'score : %f'%(score))
    result_img.save(out_file_name)
    all_img.append(result_img)
    
    # show result
    
    displayer = Image_Displayer(all_img,score,best)

    
    

if __name__ == "__main__":
    main()



Best score : 4157 Best path = [9, 30, 20, 79, 31, 19, 14, 8, 43, 38, 23, 63, 1, 90, 97, 49, 74, 83, 64, 62, 75, 80, 85, 28, 4, 42, 57, 59, 82, 77, 66, 44, 69, 60, 87, 33, 81, 7, 46, 70, 24, 89, 29, 17, 27, 91, 40, 98, 25, 2, 12, 93, 6, 45, 37, 58, 72, 95, 76, 55, 94, 32, 15, 71, 53, 13, 11, 52, 47, 65, 48, 51, 3, 84, 54, 78, 26, 41, 61, 16, 36, 18, 22, 67, 88, 96, 10, 34, 21, 73, 86, 56, 39, 35, 0, 68, 5, 92, 99, 50]
